# Importing Libraries

In [1]:
import pandas as pd
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_experimental.tools import PythonAstREPLTool
from langchain_community.document_loaders import PyPDFLoader

import os
from key import openai_api_key

os.environ['OPENAI_API_KEY'] = openai_api_key


import warnings
warnings.simplefilter(action='ignore')

# Importing Dataset

In [ ]:
tecniche = pd.read_csv('tecniche_finale.csv')
ordini_galattici = pd.read_csv('Hackapizza Dataset\Misc\ordini.csv')
licenze = pd.read_csv('Hackapizza Dataset\Misc\licenze.csv')
menus = pd.read_csv('Menus to Use\dataset_finale_menu_licenze_tecniche.csv')

# Agent Creation

## Formattatore

In [276]:
# Model
formattatore_llm = ChatOpenAI(temperature=.3, model='gpt-4o')

# Prompt
formattatore_prompt = PromptTemplate(
    template='''
    Ruolo:
    Sei il Formattatore, il tuo compito è ricevere una domanda dall'utente e fornire istruzioni dettagliate all’Assistente su come effettuare query sui dataset disponibili.
    
    Dataset a disposizione dell'Assistente:
    
    menus: Contiene i piatti disponibili con dettagli su ristorante, chef, pianeta, tecniche di preparazione, licenze necessarie, ingredienti e ID del piatto.
    tecniche: Contiene informazioni sulle tecniche di cucina del famoso chef Sirius Cosmo, è uno chef non una tecnica o un piatto, suddivise in tre categorie: Preparazione, Cottura e Avanzate. Ogni tecnica ha un nome, una breve descrizione e il grado richiesto per eseguirla.
    ordini_galattici: Contiene restrizioni alimentari per tre ordini (Ordine della Galassia di Andromeda, Ordine dei Naturalisti, Ordine degli Armonisti).
    abilita: Contiene informazioni sulle abilità richieste per cucinare determinati piatti, con nome, sigla e livelli disponibili.
    
    Regole fondamentali:
    
    Esiste sempre almeno un piatto che soddisfa la richiesta dell'utente.
    La tua risposta deve essere strutturata in step chiari e sequenziali, indicando quali operazioni l'Assistente deve eseguire sui dataset.
    Se la domanda fa riferimento a tecniche, licenze o restrizioni alimentari, devi identificare i dataset rilevanti e fornire i criteri di filtraggio adeguati.
    L’output finale dell’Assistente sarà una lista ordinata di ID dei piatti che soddisfano la richiesta.
    Esempi di istruzioni:
    
    Domanda: "Quali piatti utilizzano almeno una tecnica di taglio e una di surgelamento?"
    Istruzioni per l’Assistente:
    
    Filtra il dataset tecniche per ottenere l'elenco delle tecniche di Taglio e Surgelamento.
    Estrai i nomi di queste tecniche.
    Filtra il dataset menus per individuare i piatti che utilizzano almeno una tecnica di ciascuna categoria.
    Restituisci gli ID dei piatti ordinati in ordine crescente.
    Domanda: "Mostrami i piatti che possono essere preparati solo da chi possiede una licenza di livello III in Psionica."
    Istruzioni per l’Assistente:
    
    Filtra il dataset abilita per individuare la licenza Psionica III.
    Filtra il dataset menus per trovare i piatti che richiedono questa licenza.
    Restituisci gli ID dei piatti ordinati in ordine crescente.
    Domanda: "Trova un piatto compatibile con l’Ordine della Galassia di Andromeda che utilizzi almeno una tecnica di fermentazione."
    Istruzioni per l’Assistente:
    
    Consulta il dataset ordini_galattici per identificare gli ingredienti vietati per l’Ordine della Galassia di Andromeda.
    Filtra il dataset menus per escludere i piatti contenenti questi ingredienti.
    Filtra il dataset tecniche per ottenere l’elenco delle tecniche di Fermentazione.
    Filtra nuovamente menus per individuare i piatti che utilizzano almeno una di queste tecniche.
    Restituisci gli ID dei piatti ordinati in ordine crescente.
    La tua risposta deve essere precisa, completa e strutturata per facilitare l'elaborazione dell’Assistente.
    L'utente non chiede di individuare un piatto dal suo nome, bensì dagli ingredienti o altre caratteristiche.


    Devi essere specifico capendo che la maggior parte delle colonne sono stringhe, dunque filtrare è complesso per l'assistente. 

    Domanda utente: {domanda_originale}
    ''',
    input_variables = ['domanda_originale']
)

# Formattatore chain
formattatore = formattatore_prompt | formattatore_llm | StrOutputParser()

## Assistente

In [356]:
# Funzione per impedire il NameError in fase di tool invocation

PythonAstREPLTool_init = PythonAstREPLTool.__init__

def PythonAstREPLTool_init_wrapper(self, *args, **kwargs):
    PythonAstREPLTool_init(self, *args, **kwargs)
    self.globals = self.locals

PythonAstREPLTool.__init__ = PythonAstREPLTool_init_wrapper

In [ ]:
# Model
assistente_llm = ChatOpenAI(temperature=.3, model='gpt-4o')

# Assistente

assistente_agent = create_pandas_dataframe_agent(
    assistente_llm,
    [tecniche, ordini_galattici, menus, tecniche],
    agent_type='openai-functions',
    extra_tools=[PythonAstREPLTool()],
    verbose=False,
    allow_dangerous_code=True,
    prefix='''
    Sei l’Assistente, il tuo compito è eseguire query su dataset pandas seguendo scrupolosamente le istruzioni fornite dal Formattatore. Alla fine, devi restituire gli ID dei piatti ordinati in ordine crescente.

    Dataset a tua disposizione:
    
    menus: Ogni riga rappresenta un piatto e include informazioni su ristorante, chef, pianeta, tecniche di preparazione, licenze richieste, ingredienti e ID del piatto.
    Path per accedere a menus: "galactic_menus.csv"
    
    tecniche: Contiene tecniche di cucina suddivise in tre categorie (Preparazione, Cottura, Avanzate), con nomi, descrizioni e grado richiesto per eseguirle.
    Path per accedere a tecniche: "tecniche_finale.csv"
    
    ordini_galattici: Contiene restrizioni alimentari per tre ordini (Ordine della Galassia di Andromeda, Ordine dei Naturalisti, Ordine degli Armonisti).
    Path per accedere a ordini_galattici: "Hackapizza Dataset\Misc\ordini.csv"

    abilita: Contiene informazioni sulle abilità necessarie per preparare determinati piatti, incluse sigle e livelli disponibili.
    Path per accedere a abilita: "Hackapizza Dataset\Misc\licenze.csv"
    
    Come operare:
    
    Leggi attentamente le istruzioni fornite dal Formattatore.
    Identifica i dataset da interrogare e i criteri di filtraggio richiesti.
    Applica le operazioni di filtraggio e selezione per ottenere il risultato corretto.
    Restituisci una lista di ID dei piatti in ordine crescente.
    Esempi di richieste che potresti ricevere:
    
    "Trova i piatti che utilizzano almeno una tecnica di Taglio e una di Surgelamento."
    
    Devi estrarre l'elenco delle tecniche di queste due categorie e filtrare i piatti che ne usano almeno una per ciascuna.
    "Individua i piatti compatibili con l'Ordine della Galassia di Andromeda, che non contengono ingredienti vietati e usano almeno una tecnica di Fermentazione."
    
    Devi escludere i piatti con ingredienti proibiti e selezionare quelli che usano una tecnica di fermentazione.
    "Trova i piatti che richiedono almeno una licenza di livello II o superiore."
    
    Devi individuare le licenze di livello II o superiore e filtrare i piatti che le richiedono.
    Regole importanti:
    
    Ricorda che Sirius Cosmo è solo uno chef. Non appare nel dataset.
    Segui esattamente le istruzioni fornite dal Formattatore.
    Usa solo i dataset disponibili e le operazioni indicate.
    Restituisci sempre una lista di ID in ordine crescente.
    Seleziona solo le colonne necessarie per ottimizzare le query.
    Il tuo obiettivo è ottenere il risultato corretto nel modo più efficiente possibile. 
    Devi sempre restituire almeno un id. Esiste sempre almeno un piatto che soddisfa la richiesta. 
    Se non trovi un piatto cerca meglio, magari correggendo possibili errori di battitura oppure ampliando la ricerca.
    Non puoi rispondere che un piatto non esiste. Esiste per forza almeno un piatto.

    Gestione degli errori di battitura negli ingredienti
    Gli ingredienti presenti nel dataset menus potrebbero contenere errori di battitura o leggere variazioni nel nome. Se un ingrediente specificato nella richiesta dell'utente non corrisponde esattamente a nessuno di quelli nel dataset, devi:

    Cercare la corrispondenza più simile nel dataset usando una misura di similarità testuale (es. confronto del numero di caratteri uguali o la distanza di Levenshtein).
    Scegliere l'ingrediente con il nome più vicino a quello richiesto, purché la differenza sia minima e non alteri significativamente il significato.
    Se ci sono più alternative valide, preferire quella con la somiglianza più alta.
    Esempio:

    L'utente chiede un piatto senza "Polvere di Crononitte", ma nel dataset trovi solo "Polvere di Crononite". In questo caso, devi trattarli come lo stesso ingrediente ed escludere i piatti che lo contengono.
    Se l'utente cerca un piatto con "Fungo Stellare" ma trovi solo "Funghi Stellari", considerali equivalenti.
    L'obiettivo è evitare di scartare piatti corretti solo a causa di piccoli errori di scrittura.
    
    Non usare altre librerie oltre pandas.
    
    Restituisci solo l'id o gli id che trovi, senza aggiungere altro. Ad esempio:
     "1"
     "3, 19"
    
    ''',
    max_iterations=5
    )

# Assistente chain
assistente = {'input':RunnablePassthrough()} | assistente_agent | (lambda x: x['output'])

## Flusso

In [6]:
questions = (pd.read_csv('Hackapizza Dataset\domande.csv')
             .domanda
             .values
             .tolist())

In [ ]:
final_answers = {}

In [ ]:
# Defining the question
question = questions[0] # inserire domanda

# Formattatore
risposta_formattatore = formattatore.invoke(
     {
          'domanda_originale':question
     }
)

# Assistente
risposta_assistente = assistente.invoke(risposta_formattatore)

# Append to list
final_answers['1'] = [risposta_assistente]
final_answers[list(final_answers.keys())[-1]]

# Submission

In [ ]:
final_answers

# Terza Run  

Questa run sarà volta al miglioramento del risultato precedente andando ad usare il dataset exploded.

## Step 1 - Uniformare la Colonna Livello

Conversione della colonna livello.  
Per ogni menù viene uniformata la colonna livello, mettendo solo numeri arabi.

In [54]:
# Funzione per ottnere il documento
def full_doc(doc):
    loaded_doc = PyPDFLoader(doc).load()
    full_doc = '\n'.join([page.page_content for page in loaded_doc])
    return full_doc

In [55]:
#Caricamento dei menù
anima_cosmica_docs = full_doc('Hackapizza Dataset/Menu/Anima Cosmica.pdf')
armonia_universale_docs = full_doc('Hackapizza Dataset/Menu/Armonia Universale.pdf')
cosmica_essenza_docs = full_doc('Hackapizza Dataset/Menu/Cosmica Essenza.pdf')
datapizza_docs = full_doc('Hackapizza Dataset/Menu/Datapizza.pdf')
eco_di_pandora_docs = full_doc('Hackapizza Dataset/Menu/Eco di Pandora.pdf')
eredita_galattica_docs = full_doc('Hackapizza Dataset/Menu/Eredita Galattica.pdf')
essenza_dell_infinito_docs = full_doc('Hackapizza Dataset/Menu/Essenza dell Infinito.pdf')
il_firmamento_docs = full_doc('Hackapizza Dataset/Menu/Il Firmamento.pdf')
l_architetto_dell_universo_docs = full_doc('Hackapizza Dataset/Menu/L Architetto dell Universo.pdf')
l_eco_dei_sapori_docs = full_doc('Hackapizza Dataset/Menu/L Eco dei Sapori.pdf')
l_equilibrio_quantico_docs = full_doc('Hackapizza Dataset/Menu/L Equilibrio Quantico.pdf')
l_essenza_cosmica_docs = full_doc('Hackapizza Dataset/Menu/L Essenza Cosmica.pdf')
l_essenza_del_multiverso_su_pandora_docs = full_doc('Hackapizza Dataset/Menu/L Essenza del Multiverso su Pandora.pdf')
l_essenza_delle_dune_docs = full_doc('Hackapizza Dataset/Menu/L Essenza delle Dune.pdf')
l_essenza_di_asgard_docs = full_doc('Hackapizza Dataset/Menu/L Essenza di Asgard.pdf')
l_etere_del_gusto_docs = full_doc('Hackapizza Dataset/Menu/L Etere del Gusto.pdf')
l_oasi_delle_dune_stellari_docs = full_doc('Hackapizza Dataset/Menu/L Oasi delle Dune Stellari.pdf')
l_universo_in_cucina_docs = full_doc('Hackapizza Dataset/Menu/L Universo in Cucina.pdf')
l_infinito_in_un_boccone_docs = full_doc('Hackapizza Dataset/Menu/L infinito in un Boccone.pdf')
le_dimensioni_del_gusto_docs = full_doc('Hackapizza Dataset/Menu/Le Dimensioni del Gusto.pdf')
le_stelle_danzanti_docs = full_doc('Hackapizza Dataset/Menu/Le Stelle Danzanti.pdf')
le_stelle_che_ballano_docs = full_doc('Hackapizza Dataset/Menu/Le Stelle che Ballano.pdf')
ristorante_quantico_docs = full_doc('Hackapizza Dataset/Menu/Ristorante Quantico.pdf')
ristorante_delle_dune_stellari_docs = full_doc('Hackapizza Dataset/Menu/Ristorante delle Dune Stellari.pdf')
sala_del_valhalla_docs = full_doc('Hackapizza Dataset/Menu/Sala del Valhalla.pdf')
sapore_del_dune_docs = full_doc('Hackapizza Dataset/Menu/Sapore del Dune.pdf')
stelle_astrofisiche_docs = full_doc('Hackapizza Dataset/Menu/Stelle Astrofisiche.pdf')
stelle_dell_infinito_celestiale_docs = full_doc('Hackapizza Dataset/Menu/Stelle dell Infinito Celestiale.pdf')
tutti_a_tarsvola_docs = full_doc('Hackapizza Dataset/Menu/Tutti a TARSvola.pdf')
universo_gastronomico_di_namecc_docs = full_doc('Hackapizza Dataset/Menu/Universo Gastronomico di Namecc.pdf')

In [57]:
# Creazione dei riassunti
summarizer_model = ChatOpenAI(temperature=0.7, model='gpt-4o')
menu_summarizer_prompt = PromptTemplate(
    template = '''
    Il tuo compito è convertire un insieme di menù in un file CSV con le seguenti colonne:

    Nome ristorante → Nome del ristorante.
    Nome chef → Nome dello chef responsabile del menù.
    Licenze ottenute con punteggio → Lista delle licenze ottenute dallo chef o dal ristorante, includendo solo quelle valide e convertendo i livelli in numeri arabi (es. "II" → 2).
    Nome piatto → Nome del piatto.
    Ingredienti del piatto → Lista di ingredienti utilizzati.
    Tecniche per cucinare il piatto → Lista delle tecniche impiegate per la preparazione del piatto.
    Pianeta → Pianeta di origine del ristorante.
    Regole di Formattazione:
    Colonna "Licenze ottenute con punteggio"

    Devono essere esclusivamente tra le seguenti:
    Psionica (P)
    Temporale (t)
    Gravitazionale (G)
    Antimateria (e+)
    Magnetica (Mx)
    Quantistica (Q)
    Luce (c)
    Livello di Sviluppo Tecnologico (LTK)
    Se i livelli delle licenze sono espressi in numeri romani, convertirli in numeri arabi.
    Esempio di riga finale:
    Anima Cosmica,Aurora Stellaris,"""Psionica 3"", ""Temporale 1"", ""Gravitazionale 1"", ""Antimateria 1"", ""Quantistica 16"", ""Luce 2"", ""Livello di Sviluppo Tecnologico 2""",Nebulosa Celestiale alla Stellaris,"""Shard di Materia Oscura"", ""Carne di Balena spaziale"", ""Carne di Mucca"", ""Teste di Idra"", ""Riso di Cassandra"", ""Biscotti della Galassia"", ""Pane di Luce"", ""Funghi dell’Etere"", ""Spezie Melange""","""Cottura a Vapore con Flusso di Particelle Isoarmoniche"", ""Cottura a Vapore Termocinetica Multipla"", ""Taglio a Risonanza Sonica Rigenerativa"", ""Cottura con Microonde Entropiche Sincronizzate""",Pandora
    Anima Cosmica,Aurora Stellaris,"""Psionica 3"", ""Temporale 1"", ""Gravitazionale 1"", ""Antimateria 1"", ""Quantistica 16"", ""Luce 2"", ""Livello di Sviluppo Tecnologico 2""",Sinfonia Cosmica ma Fatta Male,"""Polvere di Pulsar"", ""Radici di Singolarità"", ""Lattuga Namecciana"", ""Baccacedro"", ""Granuli di Nebbia Arcobaleno"", ""Funghi dell’Etere"", ""Erba Pipa""","""Affumicatura a Stratificazione Quantica"", ""Cottura a Forno Dinamico Inversionale""",Pandora

    Non utilizzare parentesi quadre per le liste.
    Numeri e Formattazione

    I livelli delle licenze devono essere numeri arabi.
    Le liste devono essere separate correttamente nel CSV senza perdere formattazione.
    
    Ecco il menù: {menu}
    ''',
    input_variables = ['menu']
)

menu_summarizer = menu_summarizer_prompt | summarizer_model | StrOutputParser() 

In [ ]:
# Creazione CSV
galaxy_menus = [
    anima_cosmica_docs, armonia_universale_docs, cosmica_essenza_docs, datapizza_docs,
    eco_di_pandora_docs, eredita_galattica_docs, essenza_dell_infinito_docs, il_firmamento_docs,
    l_architetto_dell_universo_docs, l_eco_dei_sapori_docs, l_equilibrio_quantico_docs, l_essenza_cosmica_docs,
    l_essenza_del_multiverso_su_pandora_docs, l_essenza_delle_dune_docs, l_essenza_di_asgard_docs,
    l_etere_del_gusto_docs, l_oasi_delle_dune_stellari_docs, l_universo_in_cucina_docs, l_infinito_in_un_boccone_docs,
    le_dimensioni_del_gusto_docs, le_stelle_danzanti_docs, le_stelle_che_ballano_docs, ristorante_quantico_docs,
    ristorante_delle_dune_stellari_docs, sala_del_valhalla_docs, sapore_del_dune_docs, stelle_astrofisiche_docs,
    stelle_dell_infinito_celestiale_docs, tutti_a_tarsvola_docs, universo_gastronomico_di_namecc_docs
]
tot_csv = 0
csvs = []

for menu in galaxy_menus:
    csv_file = menu_summarizer.invoke({'menu':menu})
    csvs.append(csv_file)
    tot_csv += 1
    print(tot_csv)

In [59]:
for x in csvs:
    print(x)
    print('-'*100)

Ecco il file CSV con le informazioni richieste:

```csv
Nome ristorante,Nome chef,Licenze ottenute con punteggio,Nome piatto,Ingredienti del piatto,Tecniche per cucinare il piatto,Pianeta
Anima Cosmica,Aurora Stellaris,"""Psionica 3"", ""Temporale 1"", ""Gravitazionale 1"", ""Antimateria 1"", ""Quantistica 16"", ""Luce 2"", ""Livello di Sviluppo Tecnologico 2""",Nebulosa Celestiale alla Stellaris,"""Shard di Materia Oscura"", ""Carne di Balena spaziale"", ""Carne di Mucca"", ""Teste di Idra"", ""Riso di Cassandra"", ""Biscotti della Galassia"", ""Pane di Luce"", ""Funghi dell’Etere"", ""Spezie Melange""","""Cottura a Vapore con Flusso di Particelle Isoarmoniche"", ""Cottura a Vapore Termocinetica Multipla"", ""Taglio a Risonanza Sonica Rigenerativa"", ""Cottura con Microonde Entropiche Sincronizzate""",Pandora
Anima Cosmica,Aurora Stellaris,"""Psionica 3"", ""Temporale 1"", ""Gravitazionale 1"", ""Antimateria 1"", ""Quantistica 16"", ""Luce 2"", ""Livello di Sviluppo Tecnologico 2""",S

## Lavoro sul File Ufficiale

In [31]:
menus = pd.read_csv('Menus to Use\official_menus.csv')

In [37]:
# Rimuovo le ""    
menus['Licenze'] = menus['Licenze ottenute con punteggio'].str.replace('"','')
menus['Ingredienti'] = menus['Ingredienti del piatto'].str.replace('"','')
menus['Tecniche'] = menus['Tecniche per cucinare il piatto'].str.replace('"','')
# Split colonne
menus['Licenze'] = menus['Licenze'].str.split(', ')
menus['Ingredienti'] = menus['Ingredienti'].str.split(', ')
menus['Tecniche'] = menus['Tecniche'].str.split(', ')

menus_split = menus.drop(columns=['Licenze ottenute con punteggio', 
                                  'Ingredienti del piatto', 
                                  'Tecniche per cucinare il piatto'])

In [40]:
# Uso explode per rendere separati tutti gli elementi
menus_exploded_lic = menus_split.explode('Licenze')
menus_exploded_lic_tec = menus_exploded_lic.explode('Tecniche')
menus_exploded_final = menus_exploded_lic_tec.explode('Ingredienti')

In [41]:
menus_exploded_final.to_csv('menus_exploded.csv', index=None)

In [42]:
menus_exploded = pd.read_csv('menus_exploded.csv')

In [43]:
# Separo la licenza dal livello
menus_exploded[['Nome Licenza', 'Livello Licenza']] = (menus_exploded.Licenze
 .str.replace(r' (\d+[\'\+]?)(?=$)', r', \1', regex=True)
 .str.split(', ', expand=True))

In [44]:
menus_complete = menus_exploded.drop(columns='Licenze')

In [45]:
menus_complete['Nome Licenza'].unique()

array(['Psionica', 'Temporale', 'Gravitazionale', 'Antimateria',
       'Quantistica', 'Luce', 'Livello di Sviluppo Tecnologico',
       'Magnetica'], dtype=object)

In [46]:
menus_complete.to_csv('menus_complete.csv', index=None)

## Aggiunta Nuovi Dataset

Correzione errori

Aggiungo la distanza dai pianeti

In [79]:
menus_complete = pd.read_csv('menus_complete.csv')
distances = pd.read_csv('Hackapizza Dataset\Misc\Distanze.csv')

In [80]:
menus_complete.shape

(36909, 8)

In [81]:
menus_complete_with_dist = (menus_complete
 .merge(distances, how='left', on='Pianeta')
 .replace('Crocevia Dimensionalev', 'Crocevia Dimensionale'))

In [82]:
menus_complete_with_dist.shape

(36909, 18)

Aggiungo le informazioni sulle licenze

In [83]:
menus_complete_with_dist_lic = (menus_complete_with_dist
 .merge(licenze, left_on='Nome Licenza', right_on='Licenza')
 .drop(columns='Licenza')
 .rename(columns={'Descrizione':'Descrizione licenza'}))

In [84]:
menus_complete_with_dist_lic.shape

(36909, 20)

Aggiungo le informazioni sulle tecniche

In [90]:
menus_complete_with_dist_lic['Tecniche'] = menus_complete_with_dist_lic['Tecniche'].str.strip()
menus_complete_with_dist_lic_tec = (menus_complete_with_dist_lic
 .merge(tecniche, left_on='Tecniche', right_on='Nome Tecnica')
 .drop(columns='Tecniche')
 .rename(columns={'Descrizione':'Descrizione tecnica'})
)

In [91]:
menus_complete_with_dist_lic_tec.shape

(36909, 24)

Aggiungo il blog

In [92]:
blog = pd.read_csv('Hackapizza Dataset\Blogpost\\blogs.csv')

In [93]:
menus_complete_with_dist_lic_tec_blog = (menus_complete_with_dist_lic_tec
 .merge(blog, left_on='Nome ristorante', right_on='Nome Ristorante', how='left')
 .drop(columns='Nome Ristorante')) 

Aggiungo id piatti

In [94]:
dishes = (pd.read_json('Hackapizza Dataset\Misc\dish_mapping.json')
          .T
          .reset_index()
          .rename(columns={'index':'Nome piatto', 0:'ID Piatto'})
          )

In [97]:
menus_complete_with_dist_lic_tec_blog['Nome piatto'] = (menus_complete_with_dist_lic_tec_blog['Nome piatto']
                                                        .replace({
                                                            'Mandragora e Radici':'Mandragola e Radici',
                                                            'Galassia a Tavola':'Galassia a Tavola: Sinfonia di Tempeste Cosmiche',
                                                            'Sinfonia Quantica dell’Oceano Interstellare':"Sinfonia Quantica dell'Oceano Interstellare",
                                                            'Sinfonia Temporale di Fenice e Xenodonte su Pane degli Abissi':"Sinfonia Temporale di Fenice e Xenodonte su Pane degli Abissi con Colata di Plasma Vitale e Polvere di Crononite",
                                                            'Sinfonia dei Ricordi Celesti 🌱':'Sinfonia dei Ricordi Celesti',
                                                            'Il Simposio degli Infiniti Ricordi 🌈':'Il Simposio degli Infiniti Ricordi',
                                                            'Sinfonia Celestiale dei Ricordi 🪐':'Sinfonia Celestiale dei Ricordi',
                                                            'Concordanza Cosmica - Sinfonia di Sapori Multidimensionali':'Concordanza Cosmica',
                                                            'Piastrella Celestiale di Gnocchi del Crepuscolo':"Piastrella Celestiale di Gnocchi del Crepuscolo con Nebulosa di Riso di Cassandra, Lacrime di Unicorno e Velo di Materia Oscura"
                                                        }))

finale = (menus_complete_with_dist_lic_tec_blog
        .merge(dishes, on='Nome piatto'))

In [99]:
final_df = (menus_complete_with_dist_lic_tec_blog
 .merge(dishes, on='Nome piatto'))

In [100]:
final_df.shape

(36909, 26)

Salvataggio dataset definitivo da utilizzare

In [101]:
final_df.to_csv('Menus to Use\dataset_finale_menu_licenze_tecniche.csv', index=None)

# Avvio Nuovo Flusso

In [106]:
ristoranti_galassia_df = pd.read_csv('Menus to Use\dataset_finale_menu_licenze_tecniche.csv')
ordini_galattici = pd.read_csv('Hackapizza Dataset\Misc\ordini.csv')
questions = pd.read_csv('Hackapizza Dataset\domande.csv').domanda.values.tolist()

In [145]:
ristoranti_galassia_df['Nome Sotto Tecnica'].unique()

array(['Vapore', 'Tecniche di Taglio', 'Forno', 'Affumicatura',
       'Saltare in Padella', 'Marinatura', 'Surgelamento', 'Impasto',
       'Bollitura', 'Grigliare', 'Sottovuoto', 'Decostruzione',
       'Sferificazione', 'Fermentazione'], dtype=object)

In [129]:
final_answers = {}

In [146]:
# Model
formattatore_llm = ChatOpenAI(temperature=.3, model='gpt-4o')

# Prompt
formattatore_prompt = PromptTemplate(
    template='''
    Ruolo:
    Sei il Formattatore. Il tuo compito è ricevere una domanda dall'utente e fornire istruzioni dettagliate all'Assistente su come effettuare query sul dataset disponibile.

    Dataset a disposizione dell'Assistente:
    Il dataset fornito contiene informazioni sui piatti disponibili, con le seguenti colonne:

    Nome ristorante
    Nome chef
    Nome piatto
    Pianeta
    Nome licenza
    Livello licenza
    distanza_da: queste colonne hanno il format distanza_da_Nomepianeta (ad esempio distanza_da_Ego)
    Sigla licenza: abbreviazione della licenza richiesta
    Descrizione licenza: cosa fa quella licenza
    Nome Tecnica Generale: una delle seguenti ('Cottura', 'Avanzate', 'Preparazione') 
    Nome Sotto Tecnica: una delle seguenti ('Vapore', 'Tecniche di Taglio', 'Forno', 'Affumicatura','Saltare in Padella', 'Marinatura', 'Surgelamento', 'Impasto','Bollitura', 'Grigliare', 'Sottovuoto', 'Decostruzione', 'Sferificazione', 'Fermentazione')
    Nome Tecnica
    Descrizione Tecnica
    ID Piatto

    
    Riassunto sugli Ordini Galattici:
    Alcuni ordini galattici impongono restrizioni alimentari. L’Assistente deve essere in grado di filtrare i piatti in base a queste limitazioni:

    Ordine della Galassia di Andromeda → Vietati ingredienti di origine sintetica e il lattosio (latte, burro, panna, ecc.).
    Ordine dei Naturalisti → Consentiti solo ingredienti vegetali e naturali, nessun derivato animale o artificiale. Nessuna trasformazione drastica, niente manipolazioni invasive.
    Ordine degli Armonisti → Solo piatti che permettono di sintonizzarsi con lo stato emotivo di chi li assapora.
    
    Regole fondamentali:
    Esiste sempre almeno un piatto che soddisfa la richiesta dell'utente.
    La tua risposta deve essere strutturata in step chiari e sequenziali, indicando quali operazioni l'Assistente deve eseguire per filtrare il dataset.
    Se la domanda fa riferimento a tecniche, licenze o ingredienti, devi individuare i criteri di filtraggio adeguati.
    L’output finale dell’Assistente sarà una lista ordinata di ID dei piatti che soddisfano la richiesta.
    Il filtraggio è complesso perché molte colonne sono stringhe; pertanto, le istruzioni devono essere specifiche e dettagliate.
    Esempi di istruzioni:
    Domanda: "Trova i piatti che utilizzano almeno una tecnica di affumicatura e contengono agrumi."
    Istruzioni per l’Assistente:

    Filtra il dataset per individuare i piatti che utilizzano almeno una tecnica di affumicatura.
    Filtra ulteriormente i risultati per selezionare solo i piatti che contengono agrumi tra gli ingredienti.
    Restituisci gli ID dei piatti ordinati in ordine crescente.
    Domanda: "Mostrami i piatti preparati su Andromeda che non richiedono licenze speciali."
    Istruzioni per l’Assistente:

    Filtra il dataset per trovare i piatti il cui pianeta di origine è Andromeda.
    Escludi i piatti che richiedono qualsiasi tipo di licenza speciale.
    Restituisci gli ID dei piatti ordinati in ordine crescente.
    Domanda: "Trova un piatto compatibile con l’Ordine dei Naturalisti che utilizzi almeno una tecnica di fermentazione."
    Istruzioni per l’Assistente:

    Escludi dal dataset i piatti che contengono ingredienti di origine animale o artificiale.
    Filtra il dataset per ottenere l’elenco delle tecniche di fermentazione.
    Seleziona i piatti che utilizzano almeno una di queste tecniche.
    Restituisci gli ID dei piatti ordinati in ordine crescente.
    Devi essere preciso e specifico nel definire i criteri di filtraggio per l’Assistente.

    Ecco un esempio delle prime due righe del dataset:

    Nome ristorante,Nome chef,Nome piatto,Pianeta,Ingredienti,Nome Licenza,Livello Licenza,distanza_da_Tatooine,distanza_da_Asgard,distanza_da_Namecc,distanza_da_Arrakis,distanza_da_Krypton,distanza_da_Pandora,distanza_da_Cybertron,distanza_da_Ego,distanza_da_Montressosr,distanza_da_Klyntar,Sigla Licenza,Descrizione licenza,Nome Tecnica Generale,Nome Sotto Tecnica,Nome Tecnica,Descrizione tecnica,Recensione,ID Piatto
    Anima Cosmica,Aurora Stellaris,Nebulosa Celestiale alla Stellaris,Pandora,"""Shard di Materia Oscura""",Psionica,3,1130.0,473.0,987.0,1227.0,626.0,0.0,847.0,317.0,413.0,731.0,P,"Livello 0: Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti.; Livello I: lettura pensiero, telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, precognizione e visione del passato fino a 5 minuti; Livello II: manipolazione della probabilità, telecinesi e teletrasporto di oggetti di massa inferiore a 20 kg, manipolazione delle forze fondamentali dellʼuniverso; Livello III: capacità di donare la coscienza e lʼintelletto ad oggetti, manipolazione della realtà circoscritta a stanze, teletrasporto senza errore in qualsiasi dimensione temporale, comunione con entità di altri piani; Livello IV: proiezione astrale, riscrittura di realtà circoscritta a piccole nazioni o asteroidi; Livello V: riscrittura di realtà di intere linee temporali o galassie. Questo livello è equivalente al Grado di influenza di livello tecnologico III (LTK III)",Cottura,Vapore,Cottura a Vapore con Flusso di Particelle Isoarmoniche,genera vapore attraverso il controllo di particelle isoarmoniche in sintonia con la struttura molecolare del cibo,,127
    Anima Cosmica,Aurora Stellaris,Nebulosa Celestiale alla Stellaris,Pandora,"""Carne di Balena spaziale""",Psionica,3,1130.0,473.0,987.0,1227.0,626.0,0.0,847.0,317.0,413.0,731.0,P,"Livello 0: Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti.; Livello I: lettura pensiero, telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, precognizione e visione del passato fino a 5 minuti; Livello II: manipolazione della probabilità, telecinesi e teletrasporto di oggetti di massa inferiore a 20 kg, manipolazione delle forze fondamentali dellʼuniverso; Livello III: capacità di donare la coscienza e lʼintelletto ad oggetti, manipolazione della realtà circoscritta a stanze, teletrasporto senza errore in qualsiasi dimensione temporale, comunione con entità di altri piani; Livello IV: proiezione astrale, riscrittura di realtà circoscritta a piccole nazioni o asteroidi; Livello V: riscrittura di realtà di intere linee temporali o galassie. Questo livello è equivalente al Grado di influenza di livello tecnologico III (LTK III)",Cottura,Vapore,Cottura a Vapore con Flusso di Particelle Isoarmoniche,genera vapore attraverso il controllo di particelle isoarmoniche in sintonia con la struttura molecolare del cibo,,127

    Devi suggerire che la ricerca deve avvenire per step.
    Ad esempio, per cercare una determinata tecnica o licenza, prima deve vedere tutte le tecniche disponibili, poi le loro descrizioni ed infine cercare quelle che meglio soddisfano la richiesta.
    Domanda utente: {domanda_originale}
    ''',
    input_variables = ['domanda_originale']
)

# Formattatore chain
formattatore = formattatore_prompt | formattatore_llm | StrOutputParser()

In [111]:
# Funzione per impedire il NameError in fase di tool invocation

PythonAstREPLTool_init = PythonAstREPLTool.__init__

def PythonAstREPLTool_init_wrapper(self, *args, **kwargs):
    PythonAstREPLTool_init(self, *args, **kwargs)
    self.globals = self.locals

PythonAstREPLTool.__init__ = PythonAstREPLTool_init_wrapper

In [148]:
# Model
assistente_llm = ChatOpenAI(temperature=.7, model='gpt-4o')

# Assistente

assistente_agent = create_pandas_dataframe_agent(
    assistente_llm,
    ristoranti_galassia_df,
    agent_type='openai-functions',
    extra_tools=[PythonAstREPLTool()],
    verbose=True,
    allow_dangerous_code=True,
    prefix='''
    Sei l’Assistente, il tuo compito è eseguire query su dataset pandas seguendo scrupolosamente le istruzioni fornite dal Formattatore. Alla fine, devi restituire gli ID dei piatti ordinati in ordine crescente.

    Dataset a tua disposizione:
    
    ristoranti_galassia_df:
    Path per accedere a menus: "Menus to Use\dataset_finale_menu_licenze_tecniche.csv"
    
    Come operare:
    
    Leggi attentamente le istruzioni fornite dal Formattatore.
    Identifica i dataset da interrogare e i criteri di filtraggio richiesti.
    Applica le operazioni di filtraggio e selezione per ottenere il risultato corretto.
    Restituisci una lista di ID dei piatti in ordine crescente.
    Esempi di richieste che potresti ricevere:
    
    "Trova i piatti che utilizzano almeno una tecnica di Taglio e una di Surgelamento."
    
    Devi estrarre l'elenco delle tecniche di queste due categorie e filtrare i piatti che ne usano almeno una per ciascuna.
    "Individua i piatti compatibili con l'Ordine della Galassia di Andromeda, che non contengono ingredienti vietati e usano almeno una tecnica di Fermentazione."
    
    Devi escludere i piatti con ingredienti proibiti e selezionare quelli che usano una tecnica di fermentazione.
    "Trova i piatti che richiedono almeno una licenza di livello II o superiore."
    
    Devi individuare le licenze di livello II o superiore e filtrare i piatti che le richiedono.
    
    Regole importanti:
    
    Ricorda che Sirius Cosmo è solo uno chef, lo scrittore del libro di cucina da cui derivano le tecniche nel dataset. Non appare nel dataset.
    Segui esattamente le istruzioni fornite dal Formattatore.
    Usa solo i dataset disponibili e le operazioni indicate.
    Restituisci sempre una lista di ID in ordine crescente.
    Seleziona solo le colonne necessarie per ottimizzare le query.
    Il tuo obiettivo è ottenere il risultato corretto nel modo più efficiente possibile. 
    Devi sempre restituire almeno un id. Esiste sempre almeno un piatto che soddisfa la richiesta. 
    Se non trovi un piatto cerca meglio, magari correggendo possibili errori di battitura oppure ampliando la ricerca.
    Non puoi rispondere che un piatto non esiste. Esiste per forza almeno un piatto.
    Utilizza le colonne di descrizione per leggere info relative alle licenze o alle tecniche, soprattutto per le domande relative agli ordini galattici.

    Gestione degli errori di battitura negli ingredienti
    Gli ingredienti presenti nel dataset menus potrebbero contenere errori di battitura o leggere variazioni nel nome. Se un ingrediente specificato nella richiesta dell'utente non corrisponde esattamente a nessuno di quelli nel dataset, devi:

    Cercare la corrispondenza più simile nel dataset usando una misura di similarità testuale (es. confronto del numero di caratteri uguali o la distanza di Levenshtein).
    Scegliere l'ingrediente con il nome più vicino a quello richiesto, purché la differenza sia minima e non alteri significativamente il significato.
    Se ci sono più alternative valide, preferire quella con la somiglianza più alta.
    Esempio:

    L'utente chiede un piatto senza "Polvere di Crononitte", ma nel dataset trovi solo "Polvere di Crononite". In questo caso, devi trattarli come lo stesso ingrediente ed escludere i piatti che lo contengono.
    Se l'utente cerca un piatto con "Fungo Stellare" ma trovi solo "Funghi Stellari", considerali equivalenti.
    L'obiettivo è evitare di scartare piatti corretti solo a causa di piccoli errori di scrittura.
    
    Non usare altre librerie oltre pandas.

    Utilizza i vari step che hai per ragiornare.
    Se non trovi un'informazione in una colonna prova cercando in un'altra ricordandoti della ricerca precedente.
    Non possono essere presenti tutti i piatti contemporaneamente, ma solo alcuni.
    Se una query ti ritorna tutti i piatti riprova.
    
    Restituisci solo l'id o gli id che trovi, senza aggiungere altro. Ad esempio:
     "1"
     "3, 19"
    
    ''',
    max_iterations=10
    )

# Assistente chain
assistente = {'input':RunnablePassthrough()} | assistente_agent | (lambda x: x['output'])

In [ ]:
# Domanda
question = questions[0]

# Formattatore
risposta_formattatore = formattatore.invoke(
     {
          'domanda_originale':question
     }
)

# Assistente
risposta_assistente = assistente.invoke(risposta_formattatore)

# Scrittura nel dizionario
final_answers['1'] = [risposta_assistente]
#final_answers[list(final_answers.keys())[-1]]
final_answers['1']